In [64]:
import sys
sys.path.append("..") # this adds to path parent directory in order to import utils file

import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle

plt.ion()
plt.show()

import utest_utils


In [65]:
%load_ext autoreload
import importlib
importlib.reload(utest_utils)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'utest_utils' from 'C:\\workspace\\ml\\celegans_iterative_clustering\\utest_utils.py'>

## Load validation df

In [66]:
validatationData = pd.read_pickle('data/horbert.pkl')
/print validatationData.shape
genes = validatationData.columns[1:]
print (genes[:3])
validatationData.head()


(118, 968)
Index(['B0207.7', 'C01H6.9', 'C03A3.3'], dtype='object')


,class,B0207.7,C01H6.9,C03A3.3,C04G2.2,C15C8.4,C16B8.4,C32B5.6,C33A11.2,C33A12.4,...,wrk-1,xrn-2,zig-1,zig-2,zig-3,zig-4,zig-5,zig-8,ztf-3,zyg-8
0,ADA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ADE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ADF,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,ADL,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,AFD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Load data df

In [67]:
df = pd.read_pickle('data/df.pkl')
outputExcelFileName = 'no_imputation_percentual_overap_validation_prediction.xlsx'
clustersFile = 'data/no_imputation_clusters.pickle'
/print df.shape
df.head()

(7603, 12355)


,aap-1,aat-1,aat-2,aat-3,aat-5,aat-6,aat-9,abf-2,abf-5,abf-6,...,D1046.18,F54H5.14,F59A7.14,F54E2.9,C17B7.15,T28A11.25,F35F10.19,C07G3.15,F54F2.14,R06F6.14
cele-001-001.CATGACTCAA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cele-001-001.AACTACGGCT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cele-001-001.GAGGCTTATT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cele-001-001.GCCTGATATA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cele-001-001.CTGATCGACC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Load cluster values

In [68]:
# filename = '../statistical_tests/output/97GMM_umap.csv'
# clusters = pd.read_csv(filename,index_col=0 )
# print(clusters.shape)
# clusters = clusters['cluster'].values

## There are 123 genes missing

In [69]:
len(genes), Counter(np.isin(genes, df.columns))

(967, Counter({False: 123, True: 844}))

In [70]:
print(list(genes[~np.isin(genes, df.columns)]))

['B0207.7', 'C01H6.9', 'C32B5.6', 'C41G7.6 ', 'C45G9.1', 'C49G9.1', 'F08B4.4 ', 'F08G12.1 ', 'F25B4.2', 'F32B6.10', 'F49H12.4 ', 'F57H12.7', 'F58B4.3 ', 'H03A11.2 ', 'PVDL', 'R02E12.4 ', 'T17H7.1', 'T27A3.1', 'W01B6.2', 'Y113G7A.15 ', 'ZK792.1 ', 'ZK856.14 ', 'ant-1.4', 'atgr-18', 'ceh-28', 'ceh-51', 'ceh-63', 'che-1', 'clec-38', 'cyp-14A3', 'djr-1.2', 'dylt-2', 'egl-17', 'gcy-13', 'gcy-2', 'grd-8', 'grl-6', 'hlh-14', 'hlh-16', 'hlh-25', 'hlh-29', 'hlh-34', 'ins-32', 'ins-35', 'ins-7', 'inx-17', 'inx-6', 'lin-32', 'lin-58', 'lsy-27', 'lsy-6', 'mab-23', 'mir-124', 'mir-241', 'mir-273', 'mir-71', 'mir-84', 'mom-2', 'mps-3', 'mps-4', 'nas-3', 'ncx-6', 'nhr-111', 'nhr-253', 'nhr-83', 'nhx-1', 'nkat-1', 'nlp-22', 'nlp-27', 'oig-3', 'pax-3', 'pin-2', 'pqn-47', 'rcn-1', 'sams-5', 'snai-2', 'snf-5', 'spp-12', 'spp-3', 'sra-7', 'sra-9', 'srab-1', 'srab-20', 'srab-23', 'srb-6', 'srd-1', 'srd-16', 'sre-11', 'sre-30', 'sre-37', 'srg-13', 'srg-36', 'srg-41', 'srg-47', 'srg-8', 'srh-10', 'srh-132', 

## Remove missing genes from validation data

In [71]:
validatationData.head()

,class,B0207.7,C01H6.9,C03A3.3,C04G2.2,C15C8.4,C16B8.4,C32B5.6,C33A11.2,C33A12.4,...,wrk-1,xrn-2,zig-1,zig-2,zig-3,zig-4,zig-5,zig-8,ztf-3,zyg-8
0,ADA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ADE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ADF,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,ADL,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,AFD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [72]:
validatationData = validatationData[np.concatenate([['class'], genes[np.isin(genes, df.columns)]])]
genes = validatationData.columns[1:]
print(f"Validation file contains {len(validatationData['class'].unique())} clusters")
validatationData.shape

Validation file contains 118 clusters


(118, 845)

In [73]:
with open(clustersFile, 'rb') as handle:
    clusters = pickle.load(handle)

predictedClusters = list(clusters.keys())
trueClusters = validatationData['class'].unique()

In [74]:
overlapMatrix = np.zeros((len(predictedClusters), len(validatationData['class'].unique())))
# Iterate through all classes and calculate their fischer pvalue coresponding to each cluster
for i, trueClass in tqdm(enumerate(trueClusters)):
    classGenes = genes[validatationData[validatationData['class'] == trueClass][genes].values.reshape(-1) == 1]
    for j, predClass in enumerate(predictedClusters):
        intersect =  np.intersect1d(classGenes, clusters[predClass])
        overlapMatrix[j, i] = int(100 *len(intersect)/len(classGenes))
        print(f'{overlapMatrix[j, i]} ', end ='')
overlapMatrix = overlapMatrix.astype(int)
overlapMatrix = pd.DataFrame(index= predictedClusters, data= overlapMatrix, columns=trueClusters)
overlapMatrix.to_excel(outputExcelFileName)

0it [00:00, ?it/s]

27.0 9.0 18.0 0.0 9.0 9.0 18.0 27.0 0.0 9.0 18.0 36.0 18.0 18.0 9.0 54.0 18.0 9.0 36.0 18.0 9.0 9.0 36.0 18.0 27.0 0.0 0.0 27.0 18.0 18.0 18.0 27.0 36.0 5.0 8.0 37.0 2.0 11.0 14.0 11.0 8.0 5.0 5.0 8.0 20.0 8.0 11.0 2.0 14.0 2.0 5.0 11.0 28.0 25.0 11.0 22.0 11.0 14.0 8.0 42.0 14.0 20.0 5.0 11.0 14.0 20.0 

2it [00:00, 13.50it/s]

10.0 8.0 33.0 10.0 8.0 4.0 10.0 5.0 6.0 6.0 9.0 9.0 8.0 8.0 8.0 6.0 10.0 4.0 20.0 28.0 22.0 10.0 18.0 6.0 13.0 8.0 25.0 10.0 28.0 6.0 8.0 16.0 22.0 8.0 6.0 39.0 8.0 6.0 5.0 13.0 4.0 2.0 5.0 10.0 9.0 9.0 8.0 9.0 8.0 10.0 3.0 13.0 29.0 31.0 6.0 15.0 8.0 12.0 8.0 24.0 6.0 24.0 3.0 8.0 14.0 23.0 

4it [00:00, 13.41it/s]

...

115it [00:07, 15.18it/s]

18.0 62.0 0.0 18.0 12.0 12.0 18.0 0.0 12.0 31.0 6.0 12.0 18.0 12.0 12.0 6.0 6.0 6.0 0.0 0.0 6.0 6.0 0.0 18.0 0.0 25.0 6.0 12.0 0.0 0.0 6.0 6.0 0.0 9.0 31.0 9.0 9.0 18.0 18.0 22.0 4.0 9.0 22.0 13.0 9.0 18.0 13.0 9.0 9.0 13.0 13.0 9.0 4.0 4.0 13.0 4.0 22.0 13.0 18.0 4.0 22.0 0.0 4.0 13.0 13.0 13.0 

117it [00:07, 15.77it/s]

11.0 22.0 0.0 11.0 0.0 88.0 11.0 0.0 0.0 11.0 0.0 0.0 0.0 11.0 0.0 0.0 22.0 44.0 0.0 0.0 0.0 11.0 11.0 33.0 0.0 11.0 0.0 11.0 0.0 33.0 44.0 0.0 0.0 

118it [00:07, 14.88it/s]


## Plot distribution of class signature values per cluster

In [ ]:
cluster_scores = []
for i in range(results.shape[0]):
    sorted_vals = np.sort(results[i])
    cluster_scores.append(sorted_vals[1]/sorted_vals[0])
    
    ## PLOT
    plt.figure(figsize = (18, 4))
    plt.title(f'-Log10(pval) for cluster {i}, Raport smallest+1 /smallest ={cluster_scores[-1]}, smallest pval : {sorted_vals[:2]}', 
              fontsize = 15)
    plt.grid()
    plt.bar(np.arange(len(results[i])), np.sort(-np.log10(results[i]))[::-1]);
    plt.tight_layout()
    
cluster_scores = np.array(cluster_scores)

In [ ]:
plt.figure(figsize = (18, 4))
plt.title(f'Log Distribution of cluster scores (smallest+1pval/smallest_pval)', 
          fontsize = 15)
plt.grid()
plt.bar(np.arange(len(cluster_scores)), np.sort(cluster_scores), log = True);
plt.tight_layout()



In [ ]:
plt.figure(figsize = (18, 4))
plt.title(f'Top 20 worst cluster scores', fontsize = 15)
plt.grid()
plt.bar(np.arange(len(cluster_scores))[:20], np.sort(cluster_scores)[:20]);
plt.tight_layout()


## We can combine global cluster scores into 2 metrics:
- the mean of cluster scores
- number of clustres with a report < 1.5

In [ ]:
mean = np.mean(cluster_scores)
mean

In [ ]:
threshold = 1.5
print(f"There are {len(cluster_scores[cluster_scores <= 1.5])} clusters with report < {threshold}" )